In [ ]:
%matplotlib notebook
import numpy as np
from math import sin, cos, degrees, radians, atan2
import os
from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:
# PARSING FILES

test_case = 2

gt_path = '../test_cases/gt_{}.csv'.format(test_case)
measure_path = '../test_cases/measure_{}.csv'.format(test_case)
check_path = '../test_cases/check_{}.csv'.format(test_case)

def parse_file(path):
    with open(path, 'r') as f:
        line = f.readline()
        keys = line.split(', ')
        for key in keys:
            key = key.split(' ')[0]

        values = []
        while (line):
            line = f.readline()
            if line == '':
                break
            tokens = line[:-1].split(', ') # split, remove trailing \n
            line_values = [float(t) for t in tokens]
            values.append(line_values)
    return np.array(values)
    
gt = parse_file(gt_path)
meas = parse_file(measure_path)
check = parse_file(check_path)  



def get_gt_states(i, gt):
    t = gt[i][0]
    ship_states = gt[i][1:6]
    asteroids_states = []
    for j in range(7):
        asteroid_states = gt[i][(6 + j * 5):(11 + j * 5)][1:]
        asteroids_states.append(asteroid_states)
        
    return t, ship_states, asteroids_states


# CONVERTING TO SPACECRAFT INERTIAL FRAME
# origin: spacecraft starting position, spacecraft initial velocity is zero!

def convert_to_spacecraft_inertial_frame(gt):
    igt = gt.copy()
    x0 = igt[0, 1]
    y0 = igt[0, 2]
    vx0 = igt[0, 3]
    vy0 = igt[0, 4]
    x_displacements = igt[:, 0] * vx0
    y_displacements = igt[:, 0] * vy0
    igt[:, 1] -= (x_displacements + x0)
    igt[:, 2] -= (y_displacements + y0)
    igt[:, 3] -= vx0
    igt[:, 4] -= vy0
    for j in range(7):
        igt[:, 7 + j * 5] -= (x_displacements + x0)
        igt[:, 8 + j * 5] -= (y_displacements + y0)
        igt[:, 9 + j * 5] -= vx0
        igt[:, 10 + j * 5] -= vy0
    return igt
igt = convert_to_spacecraft_inertial_frame(gt)


In [ ]:
def plot_gt_step(i, gt):

    fig = plt.figure()
    ax = fig.add_subplot(111)
    t, ship_states, asteroids_states = get_gt_states(i, gt)
    mrkr = mpl.markers.MarkerStyle(marker='d')
    mrkr._transform = mrkr.get_transform().rotate_deg(degrees(ship_states[-1]))
    ax.scatter(ship_states[0], ship_states[1], marker = mrkr, s=100)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim([0, 1000])
    ax.set_ylim([0, 1000])
    for i, asteroid in enumerate(asteroids_states):
        ax.plot(asteroid[0], asteroid[1], 'ko')
        ax.text(asteroid[0]+2, asteroid[1]+2, str(i))
plot_gt_step(245, gt)

In [ ]:
def plot_gt_traces(gt, inertial=False, estimated_traces=None, test_id=None, spacecraft_trace=None, show_asteroids=True):

    color_list = ['lightcoral', 'maroon', 'turquoise', 'slateblue', 'cornflowerblue',
                 'teal', 'blueviolet', 'olivedrab', 'deeppink']
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(gt[:,1], gt[:,2], 'k--')
    ax.set_aspect('equal', adjustable='box')
    if inertial:
        ax.set_xlim([-500, 500])
        ax.set_ylim([-500, 500])
    else:
        ax.set_xlim([0, 1000])
        ax.set_ylim([0, 1000])
    for j in range(7):
        if test_id is None and show_asteroids:
            ax.plot(gt[:, 7 + j * 5], gt[:, 8 + j * 5], color = color_list[j])
        elif test_id == j and show_asteroids:
            ax.plot(gt[:, 7 + j * 5], gt[:, 8 + j * 5], color = color_list[j])
        
    if inertial and estimated_traces is not None:
        for j, trace in enumerate(estimated_traces):
            tr = np.array(trace)
            if test_id is None and show_asteroids:
                ax.plot(tr[:, 0], tr[:, 1], '--', color = color_list[j])
            elif test_id==j and show_asteroids:
                ax.plot(tr[:, 0], tr[:, 1], '--', color = color_list[j])
    
    if spacecraft_trace is not None:
        tr = np.array(spacecraft_trace) 
        ax.plot(tr[:, 0], tr[:, 1], 'k--', alpha=0.5)
                
    
    
plot_gt_traces(igt, True, show_asteroids=True)




In [ ]:
plot_gt_traces(gt)    

In [ ]:

# GT
#      0,      1,     2,        3,        4,           5,  6,     7,     8,        9,       10, 11,    12,    13, 
#Time (s), x (m), y (m), vx (m/s), vy (m/s), theta (rad), ID, x (m), y (m), vx (m/s), vy (m/s), ID, x (m), y (m), ...

# MEASURE
#      0,                 1,  2,        3,             4,  5,         6,            7,  8,        9,            10,
#Time (s), Sun Bearing (px), ID, Range (m), Bearing (px), ID, Range (m), Bearing (px), ID, Range (m), Bearing (px), 

In [ ]:
# Calculating Sun Angle Errors

fig = plt.figure()
ax = fig.add_subplot(111)
# ax.plot(gt[:, 5], -meas[:, 1], '.')
# ax.plot(gt[:, 5], (-meas[:, 1] + 180) % 360 - 180, '.')

error = ((-meas[:, 1] + 180) % 360 - 180) - np.degrees(gt[:, 5])
ax.plot(error)
print('Sun angle has {} degrees mean error with +/- {} standard deviation.'.format(error.mean(), error.std()))
print('minimum error {} degrees, maximum error {} degrees.'.format(abs(error).min(), abs(error).max()))

In [ ]:
# GT
#      0,      1,     2,        3,        4,           5,  6,     7,     8,        9,       10, 11,    12,    13, 
#Time (s), x (m), y (m), vx (m/s), vy (m/s), theta (rad), ID, x (m), y (m), vx (m/s), vy (m/s), ID, x (m), y (m), ...


# MEASURE
#      0,                 1,  2,        3,             4,  5,         6,            7,  8,        9,            10,
#Time (s), Sun Bearing (px), ID, Range (m), Bearing (px), ID, Range (m), Bearing (px), ID, Range (m), Bearing (px), 

In [ ]:
def get_asteroid_meas(step, a_id, meas):
    m_range = meas[step, 3 + a_id * 3]
    m_bearing = meas[step, 4 + a_id * 3]
    return m_range, m_bearing
r, b = get_asteroid_meas(240, 2, meas)


def relative_pos(m_range, m_bearing, sun_angle):
    return m_range*cos(radians(m_bearing - sun_angle)), m_range*sin(radians(m_bearing - sun_angle))


# assume knowledge of the ship
def simulation1(meas, igt):
    
    asteroid_traces = [[] for _ in range(7)]
    
    for step in range(meas.shape[0]):
        sun = meas[step, 1]
#         sun = -degrees(igt[step, 5])
        for asteroid_id in range(7):
            r, b = get_asteroid_meas(step, asteroid_id, meas)
            xr, yr = relative_pos(r, b, sun)
            x = xr + igt[step, 1]
            y = yr + igt[step, 2]
            asteroid_traces[asteroid_id].append([x, y])
     
    return asteroid_traces


traces = simulation1(meas, igt)    

In [ ]:
plot_gt_traces(igt, True, traces)

In [ ]:
plot_gt_traces(igt, True, traces)

In [ ]:
class KalmanFilter:
    def __init__(self, x0, P0, A, B, H, Q, R):
        self.x = x0
        self.P = P0
        self.A = A
        self.B = B
        self.H = H
        self.Q = Q
        self.R = R
        
    def update(self, z, u=None, update = True):
        
        # predict
        x_priori = self.A @ self.x
        if B is not None:
            x_priori += self.B @ u
        P_priori = self.A @ self.P @ self.A.transpose() + self.Q
        
        # update
        K = P_priori @ self.H.transpose() @ np.linalg.inv(self.H @ P_priori @ self.H.transpose() + self.R)
        residual = z - self.H @ x_priori
                
        x_posteriori = x_priori + K @ (residual)
        P_posteriori = (np.eye(4) - K @ self.H) * P_priori
        
        if update:
            self.x = x_posteriori
            self.P = P_posteriori
            
        return x_posteriori, np.linalg.norm(residual)
        
        
        
    
x0 = np.zeros([4, 1]) 
P0 = np.eye(4) * 0.1
dt = 0.2
A = np.eye(4)
A[0, 2] = dt
A[1, 3] = dt
B = None
H = np.hstack([np.eye(2), np.zeros([2, 2])])
Q = np.eye(4) * 0.1
R = np.eye(2) * 0.1

# Simple KF test

kf = KalmanFilter(x0, P0, A, B, H, Q, R)

for j in range(20):
    z = np.array([[j/5], [-j/5]])
    out, residual = kf.update(z)
print(out[3])


In [ ]:
# assume knowledge of the ship, track with Kalman filters!!!

x0 = None
# P0 = np.eye(4) * 10
P0 = np.diag([10, 10, 10, 10])
dt = 0.2
A = np.eye(4)
A[0, 2] = dt
A[1, 3] = dt
B = None
H = np.hstack([np.eye(2), np.zeros([2, 2])])
Q = np.eye(4) * 0.1
R = np.eye(2) * 20

speed_estimates = []
pos_errors = []
test_id = None

def simulation2(meas, igt):
    
    asteroid_traces = [[] for _ in range(7)]
    
    kalman_filters = []
    
    for step in range(meas.shape[0]):
        sun = meas[step, 1]
#         sun = -degrees(igt[step, 5])
        for asteroid_id in range(7):
            r, b = get_asteroid_meas(step, asteroid_id, meas)
            xr, yr = relative_pos(r, b, sun)
            x = xr + igt[step, 1]
            y = yr + igt[step, 2]
            
            if step == 0:
                x0 = np.array([[x],[y], [0], [0]])
                kalman_filters.append(KalmanFilter(x0, P0, A, B, H, Q, R))
                asteroid_traces[asteroid_id].append([x, y])
            else:
                z = np.array([[x],[y]])
                x_posteriori, res = kalman_filters[asteroid_id].update(z)
                asteroid_traces[asteroid_id].append([x_posteriori[0], x_posteriori[1]])
                if test_id is not None and asteroid_id == test_id:
                    speed_estimates.append(x_posteriori[2])
                    pos_errors.append(np.linalg.norm([igt[step, 7 + test_id*5] - x_posteriori[0],
                                                      igt[step, 8 + test_id*5] - x_posteriori[1]]))
            
            
     
    return asteroid_traces


traces = simulation2(meas, igt)    
plot_gt_traces(igt, True, traces, test_id)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(speed_estimates)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(pos_errors)
ax.plot([0, 250], [1.5, 1.5], '--r')
ax.set_ylim([0, 10])


In [ ]:
class Spaceship:
    def __init__(self, sun_position, dt=0.2, thrust=5):
        self.x = np.array([[0, 0, 0, 0, -radians(sun_position)]]).transpose()
        self.A = np.eye(5)
        self.A[0, 2] = dt
        self.A[1, 3] = dt
        self.dt = dt
        self.thrust = thrust
        
    def propagate(self, sun_position, accelerate=False, update=False):
        x_prop = self.A @ self.x
        if accelerate:
            Bu = np.zeros([5, 1])
            Bu[0, 0] = 0.5 * self.thrust * self.dt * self.dt * cos(self.x[4, 0])
            Bu[1, 0] = 0.5 * self.thrust * self.dt * self.dt * sin(self.x[4, 0])
            Bu[2, 0] = self.thrust * self.dt * cos(self.x[4, 0])
            Bu[3, 0] = self.thrust * self.dt * sin(self.x[4, 0])
            x_prop += Bu
            
            
        x_prop[4, 0] = -radians(sun_position)
        
        if update:
            self.x = x_prop
            
        return x_prop
                                                   
        
        
ship = Spaceship(0)
ship.propagate(0, True, True)
ship.propagate(0, False, True)
ship.propagate(180, True, True)
ship.propagate(180, True, True)
ship.propagate(180, False, False)
ship.propagate(180, True, True)
ship.propagate(180, False, True)
ship.propagate(180, False, True)[4, 0]




In [ ]:
ship = Spaceship(0)
ship.propagate(0, True, True)
ship.propagate(0, False, True)
ship.propagate(180, True, True)
ship.propagate(180, True, True)
ship.propagate(180, True, True)
ship.propagate(180, False, True)
ship.propagate(180, False, True)[4, 0]

In [ ]:
# Live scenario!!! Woohoo!!! Gooo spaceship gooooo!

x0 = None
# P0 = np.eye(4) * 10
P0 = np.diag([10, 10, 10, 10])
dt = 0.2
A = np.eye(4)
A[0, 2] = dt
A[1, 3] = dt
B = None
H = np.hstack([np.eye(2), np.zeros([2, 2])])
Q = np.eye(4) * 0.1
R = np.eye(2) * 20

speed_estimates = []
pos_errors = []
test_id = None

def simulation3(meas):
    
    asteroid_traces = [[] for _ in range(7)]
    asteroid_bearings = [[] for _ in range(7)]
    asteroid_ranges = [[] for _ in range(7)]

    spacecraft_trace = []
    
    kalman_filters = []
    
    spaceship = None
    
    for step in range(meas.shape[0]):
        sun = meas[step, 1]
        
        if spaceship is None:
            spaceship = Spaceship(sun)
        
        no_acc_residuals = 0.0
        acc_residuals = 0.0
        
        ss_noacc = spaceship.propagate(sun, accelerate=False, update=False)
        x_noacc, y_noacc = ss_noacc[0,0], ss_noacc[1,0]
        ss_acc = spaceship.propagate(sun, accelerate=True, update=False)
        x_acc, y_acc = ss_acc[0,0], ss_acc[1,0]
        
        for asteroid_id, kf in enumerate(kalman_filters):  # to skip initialization step
            r, b = get_asteroid_meas(step, asteroid_id, meas)
            xr, yr = relative_pos(r, b, sun)
            xa_noacc = xr + x_noacc
            ya_noacc = yr + y_noacc
            
            xa_acc = xr + x_acc
            ya_acc = yr + y_acc
            
            _, res_noacc = kalman_filters[asteroid_id].update(np.array([[xa_noacc],[ya_noacc]]), update=False)
            no_acc_residuals += res_noacc

            _, res_acc = kalman_filters[asteroid_id].update(np.array([[xa_acc],[ya_acc]]), update=False)
            acc_residuals += res_acc
            
        if (acc_residuals < no_acc_residuals):
            xss, yss = x_acc, y_acc
            ss_state = spaceship.propagate(sun, accelerate=True, update=True)
            spacecraft_trace.append([ss_state[0], ss_state[1]])
        else:

            xss, yss = x_noacc, y_noacc
            ss_state = spaceship.propagate(sun, accelerate=False, update=True)
            spacecraft_trace.append([ss_state[0], ss_state[1]])
        
        for asteroid_id in range(7):
            r, b = get_asteroid_meas(step, asteroid_id, meas)
            xr, yr = relative_pos(r, b, sun)
            
            x = xr + xss
            y = yr + yss
            
            if step == 0:
                x0 = np.array([[x],[y], [0], [0]])
                kalman_filters.append(KalmanFilter(x0, P0, A, B, H, Q, R))
                asteroid_traces[asteroid_id].append([x, y])
            else:
                z = np.array([[x],[y]])
                x_posteriori, res = kalman_filters[asteroid_id].update(z)
                asteroid_traces[asteroid_id].append([x_posteriori[0], x_posteriori[1]])
                
                # store bearing and range here
                rel_x = x_posteriori[0] - xss
                rel_y = x_posteriori[1] - yss
                ss_heading = -radians(sun)
                
                asteroid_ranges[asteroid_id].append(np.linalg.norm([rel_x, rel_y]))
                asteroid_bearings[asteroid_id].append(atan2(rel_y, rel_x) - ss_heading)
                
                if test_id is not None and asteroid_id == test_id:
                    speed_estimates.append(x_posteriori[2])
                    pos_errors.append(np.linalg.norm([igt[step, 7 + test_id*5] - x_posteriori[0],
                                                      igt[step, 8 + test_id*5] - x_posteriori[1]]))
            
                
    return asteroid_traces, spacecraft_trace, np.array(asteroid_ranges).transpose(), np.array(asteroid_bearings).transpose()


asteroid_traces, spacecraft_trace, asteroid_ranges, asteroid_bearings = simulation3(meas)    
# plot_gt_traces(igt, True, asteroid_traces, test_id, spacecraft_trace)
# plot_gt_traces(igt, True, None, None, spacecraft_trace, show_asteroids=False)
plot_gt_traces(igt, True, asteroid_traces, None, spacecraft_trace, show_asteroids=True)

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(speed_estimates)

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(pos_errors)
# ax.plot([0, 250], [1.5, 1.5], '--r')
# ax.set_ylim([0, 10])


In [ ]:
np.array(spacecraft_trace)[:, 0].shape

In [ ]:
# GT
#      0,      1,     2,        3,        4,           5,  6,     7,     8,        9,       10, 11,    12,    13, 
#Time (s), x (m), y (m), vx (m/s), vy (m/s), theta (rad), ID, x (m), y (m), vx (m/s), vy (m/s), ID, x (m), y (m), ...


# MEASURE
#      0,                 1,  2,        3,             4,  5,         6,            7,  8,        9,            10,
#Time (s), Sun Bearing (px), ID, Range (m), Bearing (px), ID, Range (m), Bearing (px), ID, Range (m), Bearing (px), 

In [ ]:
def convert_to_relative(gt):
    rgt = gt.copy()
    
    gt_ranges = [[] for _ in range(7)]
    gt_bearings = [[] for _ in range(7)]
    
    for j in range(7):
        for step in range(rgt.shape[0]):
            x_rel = rgt[step, 7 + 5*j] - rgt[step, 1]
            y_rel = rgt[step, 8 + 5*j] - rgt[step, 2]
            gt_ranges[j].append(np.linalg.norm([x_rel, y_rel]))
            gt_bearings[j].append(atan2(y_rel, x_rel) - rgt[step, 5])
    
    
    return np.array(gt_ranges).transpose(), np.array(gt_bearings).transpose()
gt_ranges, gt_bearings = convert_to_relative(gt)

In [ ]:
def  plot_polars(gt_ranges, gt_bearings, a_ranges, a_bearings):
    
    color_list = ['lightcoral', 'maroon', 'turquoise', 'slateblue', 'cornflowerblue',
                 'teal', 'blueviolet', 'olivedrab', 'deeppink']
    
    fig = plt.figure()
    for i in range(7):
        plt.polar(gt_bearings[:, i], gt_ranges[:, i], color=color_list[i])
        plt.polar(a_bearings[:, i], a_ranges[:, i], '*', color=color_list[i])
    
plot_polars(gt_ranges, gt_bearings, asteroid_ranges, asteroid_bearings)

In [ ]:
def  plot_cartesian(gt_ranges, gt_bearings, a_ranges, a_bearings, index=None):
    
    color_list = ['lightcoral', 'maroon', 'turquoise', 'slateblue', 'cornflowerblue',
                 'teal', 'blueviolet', 'olivedrab', 'deeppink']
    
    fig = plt.figure()
    for i in range(7):
        if index is None or i==index:
            x = gt_ranges[:, i] * np.cos(gt_bearings[:, i])
            y = gt_ranges[:, i] * np.sin(gt_bearings[:, i])
            plt.plot(x, y, color=color_list[i])
            xa = a_ranges[:, i] * np.cos(a_bearings[:, i])
            ya = a_ranges[:, i] * np.sin(a_bearings[:, i])
            plt.plot(xa, ya, '--', color=color_list[i])

plot_cartesian(gt_ranges, gt_bearings, asteroid_ranges, asteroid_bearings, None)


In [ ]:
# Let's create a moving average filter
a = (np.arange(400)+ 180) % 360 - 180
# a = np.ones(100)*20

a += np.random.randint(-2, 2, a.shape)

n = 5


sin_vals = []
cos_vals = []

def put_new_value(val):
    rad_val = radians(val)
    sin_vals.append(sin(rad_val))
    cos_vals.append(cos(rad_val))
    
    
def get_smooth():
    while(len(sin_vals) > n):
        _ = sin_vals.pop(0)
        
    while(len(cos_vals) > n):
        _ = cos_vals.pop(0)
        
    sin_avg = np.average(sin_vals)
    cos_avg = np.average(cos_vals)
    
    
    return degrees(atan2(sin_avg, cos_avg))
    

results = []
    
for val in a:
    put_new_value(val)
    smooth = get_smooth()
    results.append(smooth)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(results)
ax.plot(a)
ax.grid(True)
    